# Fixed effects and Blei/Wang's deconfounder

Can we use the approach described in [The Blessings of Multiple Causes](https://arxiv.org/abs/1805.06826) to work in the fixed effects set up.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.decomposition import PCA, FactorAnalysis

### Set up data

Find the impact of "education level" (observed) on income in the presence of unobserved bias (a person's ability). For each individual, we observe their education level and income for three years. Each person has an independent ability level that is time invariant. See page 246 of the [Causal Inference Mixtape](http://scunning.com/cunningham_mixtape.pdf) for a picture of the DAG.

In [2]:
np.random.seed(6429)
n_people = 100
n_years = 3
people = pd.DataFrame.from_dict(
    {
        "person":np.arange(n_people),
        "ability":np.random.poisson(lam=5, size=n_people),
    }
)
people["person"] = people["person"].astype("category")
people["1990"] = np.random.poisson(lam=(people["ability"]), size=n_people)
people["1991"] = people["1990"] + np.random.poisson(lam=(people["ability"]), size=n_people)
people["1992"] = people["1991"] + np.random.poisson(lam=(people["ability"]), size=n_people)

In [3]:
people.head()

,person,ability,1990,1991,1992
0,0,3,5,9,13
1,1,3,3,5,5
2,2,3,4,7,12
3,3,5,6,10,14
4,4,3,1,7,10


### Factor analysis for the deconfounder

In this set up, the multiple causes are the education levels for each person in each of the three years. Since I'm lazy, I assume that one hidden dimension is good enough since we know there is only one unobserved dimension.

In [4]:
X = people[["1990", "1991", "1992"]]
pca = PCA(n_components=1)
factor = FactorAnalysis(n_components=1)
people["pca"] = pca.fit_transform(X)
people["factor"] = factor.fit_transform(X)
people["noise"] = np.random.normal(0, 1, n_people)

Melt the data to get it into a format for fixed effects and add income (potentially with noise)

In [5]:
noise_sd = 0
people_melted = (people
          .melt(id_vars=['person', 'ability', 'pca', 'noise', 'factor'])
          .rename({"variable":"year", "value":"education_level"}, axis=1)
          .eval("income = 4 + 3 * education_level + ability")
          .assign(income = lambda df: df["income"] + np.random.normal(0,noise_sd,n_people*n_years))
         )

In [6]:
people_melted.head()

,person,ability,pca,noise,factor,year,education_level,income
0,0,3,-0.946288,-0.269890,-0.069749,1990,5,22.0
1,1,3,-10.041739,0.758900,-1.045307,1990,3,16.0
2,2,3,-3.073523,1.209119,-0.455291,1990,4,19.0
3,3,5,0.644646,0.612017,0.154519,1990,6,27.0
4,4,3,-5.430591,0.116214,-0.610194,1990,1,10.0


### If we include ability, we should recover everything

In [7]:
mod = smf.ols("income ~ education_level + ability", data=people_melted)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.220e+31
Date:                Tue, 25 Jun 2019   Prob (F-statistic):               0.00
Time:                        07:57:23   Log-Likelihood:                 8811.0
No. Observations:                 300   AIC:                        -1.762e+04
Df Residuals:                     297   BIC:                        -1.760e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.0000   6.78e-15    5

### Since we don't observe ability, a naive regression will be biased

In [8]:
mod = smf.ols("income ~ education_level", data=people_melted)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 4.695e+04
Date:                Tue, 25 Jun 2019   Prob (F-statistic):               0.00
Time:                        07:57:23   Log-Likelihood:                -561.42
No. Observations:                 300   AIC:                             1127.
Df Residuals:                     298   BIC:                             1134.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.2770      0.163     

### Fixed effects recovers the truth

In [14]:
mod = smf.mixedlm("income ~ education_level", data=people_melted, groups=people_melted["person"])
# mod = smf.ols("income ~ education_level + person", data=people_melted)
res = mod.fit()
print(res.summary())

             Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   income   
No. Observations:     300       Method:               REML     
No. Groups:           100       Scale:                0.0000   
Min. group size:      3         Likelihood:           1710.0173
Max. group size:      3         Converged:            Yes      
Mean group size:      3.0                                      
---------------------------------------------------------------
                Coef. Std.Err.      z       P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept       8.800    0.109       80.969 0.000  8.587  9.013
education_level 3.000    0.000 15192666.200 0.000  3.000  3.000
Group Var       1.181 7595.699                                 



### The deconfounder using PCA or factor analysis should also work

In [10]:
mod = smf.ols("income ~ education_level + pca", data=people_melted)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 4.748e+04
Date:                Tue, 25 Jun 2019   Prob (F-statistic):               0.00
Time:                        07:57:24   Log-Likelihood:                -455.72
No. Observations:                 300   AIC:                             917.4
Df Residuals:                     297   BIC:                             928.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           8.8180      0.145     

In [11]:
mod = smf.ols("income ~ education_level + factor", data=people_melted)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 4.157e+04
Date:                Tue, 25 Jun 2019   Prob (F-statistic):               0.00
Time:                        07:57:24   Log-Likelihood:                -475.60
No. Observations:                 300   AIC:                             957.2
Df Residuals:                     297   BIC:                             968.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           8.6905      0.154     

### Using a unit-specific noise term shouldn't work

In [12]:
mod = smf.ols("income ~ education_level + noise", data=people_melted)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 2.340e+04
Date:                Tue, 25 Jun 2019   Prob (F-statistic):               0.00
Time:                        07:57:24   Log-Likelihood:                -561.41
No. Observations:                 300   AIC:                             1129.
Df Residuals:                     297   BIC:                             1140.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.2759      0.164     